Finding question ids with lowest agreement

In [1]:
import pandas as pd
import numpy as np


joe_final = pd.read_csv('./final_joe.csv')

sinan_final = pd.read_csv('./final_sinan.csv')

sam_final = pd.read_csv('./final_sam.csv')

In [2]:
sinan_coord = sinan_final[["x_coordinate", "y_coordinate"]]


sam_coord = sam_final[["x_coordinate", "y_coordinate"]]


joe_coord = joe_final[["x_coordinate", "y_coordinate"]]

sinan_coord['rater_id'] = 1
sam_coord['rater_id'] = 2
joe_coord['rater_id'] = 3

sinan_coord['question_id'] = sinan_coord.index + 1
sam_coord['question_id'] = sam_coord.index + 1
joe_coord['question_id'] = joe_coord.index + 1

sn_df = pd.merge(sinan_coord, sam_coord, on='question_id', suffixes=('_sinan', '_sam'))
final_df = pd.merge(sn_df, joe_coord, on='question_id')
final_df.rename(columns={'x_coordinate': 'x_coordinate_joe', 'y_coordinate': 'y_coordinate_joe'}, inplace=True)

C:\Users\joely\AppData\Local\Temp\ipykernel_50660\2529468228.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinan_coord['rater_id'] = 1
C:\Users\joely\AppData\Local\Temp\ipykernel_50660\2529468228.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sam_coord['rater_id'] = 2
C:\Users\joely\AppData\Local\Temp\ipykernel_50660\2529468228.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

Define number of worst songs we want to retrieve.

In [3]:
N = 500

In [5]:
def custom_distance_func(a, b):
    return abs(a - b) / 2

Now we need to compute the distance between each song of each pair of annotators. We then add the values to get a total distance between annotators. This can then be stored on a new column in the dataframe, and the top N worst songs (in terms of agreement) can be retrieved. 

In [6]:
def compute_distances(row):
    sinan_sam = (custom_distance_func(row['x_coordinate_sinan'], row['x_coordinate_sam']),
                               custom_distance_func(row['y_coordinate_sinan'], row['y_coordinate_sam']))
    sinan_joe = (custom_distance_func(row['x_coordinate_sinan'], row['x_coordinate_joe']),
                               custom_distance_func(row['y_coordinate_sinan'], row['y_coordinate_joe']))
    sam_joe = (custom_distance_func(row['x_coordinate_sam'], row['x_coordinate_joe']),
                             custom_distance_func(row['y_coordinate_sam'], row['y_coordinate_joe']))
    
    total_distance = sinan_sam[0] + sinan_sam[1] + sinan_joe[0] + sinan_joe[1] + sam_joe[0] + sam_joe[1]
    return total_distance

final_df['total_distance'] = final_df.apply(compute_distances, axis=1)

#Get the top N songs with the worst agreement
top_N_worst = final_df.nlargest(N, 'total_distance')

print(top_N_worst[['question_id', 'total_distance']])

     question_id  total_distance
121          122             3.0
532          533             3.0
182          183             3.0
295          296             2.9
186          187             2.8
..           ...             ...
36            37             1.2
44            45             1.2
123          124             1.2
125          126             1.2
141          142             1.2

[500 rows x 2 columns]


In [7]:
bad_ids = top_N_worst.index.to_list()

Calculating genres of lowest agreed 100 songs for analysis

In [8]:
data = pd.read_csv('final_joe.csv')

tags = data['tag'].to_list()

rap_score = 0
rock_score = 0
rb_score = 0
pop_score = 0
country_score = 0


for id in bad_ids:
    if tags[id] == 'rap':
        rap_score += 1
    if tags[id] == 'rock':
        rock_score += 1
    if tags[id] == 'rb':
        rb_score += 1
    if tags[id] == 'pop':
        pop_score += 1
    if tags[id] == 'country':
        country_score += 1

print(f"Rap score: {rap_score}, Rb score: {rb_score}, rock score: {rock_score}, pop score: {pop_score}, country score: {country_score}")

Rap score: 90, Rb score: 88, rock score: 101, pop score: 117, country score: 104


This is after we reannotated. I am changing the reannotated values to 0 in dataset 
This will show up as having full agreement, which we now do.
So we can calculate new agreement score after reannotation

In [9]:
if N == 100:
    joe_x = joe_final['x_coordinate'].to_list()
    joe_y = joe_final['y_coordinate'].to_list()
    sam_x = sam_final['x_coordinate'].to_list()
    sam_y = sam_final['y_coordinate'].to_list()
    sinan_x = sinan_final['x_coordinate'].to_list()
    sinan_y = sinan_final['y_coordinate'].to_list()
    for id in bad_ids:
        joe_x[id] = 0
        joe_y[id] = 0
        sam_x[id] = 0
        sam_y[id] = 0
        sinan_x[id] = 0
        sinan_y[id] = 0

    joe_final['x_coordinate'] = joe_x
    joe_final['y_coordinate'] = joe_y
    sam_final['x_coordinate'] = sam_x
    sam_final['y_coordinate'] = sam_y
    sinan_final['x_coordinate'] = sinan_x
    sinan_final['y_coordinate'] = sinan_y    

In [10]:
if N == 100:
    joe_final.to_csv('joe_new_agreed.csv')
    sam_final.to_csv('sam_new_agreed.csv')
    sinan_final.to_csv('sinan_new_agreed.csv')

Dropping all bad ids to find the agreement scores of the top x of the dataset.

In [11]:
joe_worst_removed = joe_final.drop(index=bad_ids)

sam_worst_removed = sam_final.drop(index=bad_ids)

sinan_worst_removed = sinan_final.drop(index=bad_ids)

In [12]:
len(joe_worst_removed)

498

In [13]:
joe_worst_removed.to_csv('joe_worst_removed.csv')

sam_worst_removed.to_csv('sam_worst_removed.csv')

sinan_worst_removed.to_csv('sinan_worst_removed.csv')

Worst 200 removed gives: 
0.5313 for x_alpha
0.3575 for y_alpha

Worst 100 removed gives: 
0.4574 for x_alpha 
0.3126 for y_alpha

Id be happy with worst 100, then if we have time, do more, the more the better. 
Worst 100 gives us moderate agreement (0.41-0.60) for x

We didnt have time to do more than 100 in the end, annotating is very time consuming 



